In [1]:
%load_ext autoreload
%autoreload 1

from dataset import Dataset, Synthetic_data

import numpy as np
from scipy.linalg import orthogonal_procrustes

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from tqdm import tqdm

from mu_dist import Mu
from sigma_dist import Sigma_Star
from gamma_dist import Gamma

from r_dist import R
from z_dist import Z
from phi_dist import Phi

from dataset_initialisation import GMM_Init

from scipy.stats import beta

In [2]:
%aimport dataset
%aimport dataset_initialisation

In [4]:
μ_1 = np.array([0.8,0.15,0.05])
μ_2 = np.array([0.4,0.45,0.15])
μ_3 = np.array([0.1,0.3,0.6])

    # [0.8018,0.2673,0.5345] 
    # [0.2673,0.8018,0.5345]
α = 2
β = 2
prior = lambda : beta.rvs(α, β)

In [5]:
from dataset_initialisation import GMM_Init

In [6]:
# class GMM_Init():

#     def __init__(self, dataset, n_components=2):
#         self.dataset = dataset
#         gmm = GaussianMixture(n_components=n_components, covariance_type='full')
#         self.fitted_gmm = gmm.fit(dataset)

#         self.cluster_centres = self.fitted_gmm.means_
#         self.cluster_covs = self.fitted_gmm.covariances_

#         self.gamma_estimates = np.array([self.gamma_scaled(cov_mat) for cov_mat in self.cluster_covs])
    

#     def mu_prior_cov_estimate(self):
#         cluster_centres = self.fitted_gmm.means_

#         print(cluster_centres, "cluster centres")

#         return np.cov(cluster_centres.T)
    
#     def gamma_scaled(self, cov_mat):
#         ν = cov_mat[-1,-1]
#         γ = cov_mat[-1,:-1] / np.sqrt(ν)
#         return γ

    
#     def gamma_prior_cov_estimate(self):
#         print("self.gamma_estimates: ", self.gamma_estimates)
#         gamma_cov_estimate = np.cov(self.gamma_estimates.T)
#         return gamma_cov_estimate


#     def sigma_prior_params_estimate(self):
#         d = len(self.cluster_covs[0])

#         sigma_star_estimates = []

#         for i, cov_mat in enumerate(self.cluster_covs):
#             sigma_star_estimates.append(cov_mat[:-1,:-1] - np.outer(self.gamma_estimates[i], self.gamma_estimates[i]))
        
#         sigma_star_estimates = np.array(sigma_star_estimates)

#         self.sigma_star_estimates = sigma_star_estimates

#         dim = len(sigma_star_estimates[0])
#         dof = d+3 # Set it to be d+3 for now
#         sigma_star_mean = np.mean(sigma_star_estimates, axis = 0)
#         scale = sigma_star_mean * (dof - (dim + 1))

#         return scale, dof

In [7]:
ds = Synthetic_data(μ_1, μ_2, μ_3, prior, N_t=1000)

In [8]:
# import matplotlib.pyplot as plt

# plt.scatter(ds.normed_embds[:,0], ds.normed_embds[:,1])

In [9]:
ds.gaussian_mm_init()

[[0.12276602 0.53247079 0.78196976]
 [0.8844881  0.08486708 0.32211674]
 [0.73748388 0.57573496 0.03567277]] cluster centres
self.gamma_estimates:  [[ 0.03146241 -0.19976272]
 [-0.10038209  0.00028987]
 [-0.04491001  0.03882339]]
mu_cov 0 det: 0.0017095304850218863
scale_mat 0 det: 0.0023409947367255086
gamma_cov 0 det: 7.188850842331236e-05
gamma cov: [[0.00438215 0.        ]
 [0.         0.01640487]]
mu_cov 1 det: 0.0017095304850218863
scale_mat 1 det: 0.0023409947367255086
gamma_cov 1 det: 7.188850842331236e-05
gamma cov: [[0.00438215 0.        ]
 [0.         0.01640487]]
mu_cov 2 det: 0.0017095304850218863
scale_mat 2 det: 0.0023409947367255086
gamma_cov 2 det: 7.188850842331236e-05
gamma cov: [[0.00438215 0.        ]
 [0.         0.01640487]]


In [10]:
def print_attributes(obj):
    attributes = dir(obj)
    attributes = [attr for attr in attributes if not attr.startswith('__') and not callable(getattr(obj, attr))]
    for attr in attributes:
        value = getattr(obj, attr)
        print(f"{attr}: {value}")

## μ variables

In [11]:
print_attributes(ds.means_vars[0])
print()
print("cov matrix det:" , np.linalg.det(ds.means_vars[0].cov))

cov: [[0.1632848  0.         0.        ]
 [0.         0.07386203 0.        ]
 [0.         0.         0.14174569]]
d: 3
k: 0
mean: [0.12276602 0.53247079 0.78196976]
prior_cov: [[0.1632848  0.         0.        ]
 [0.         0.07386203 0.        ]
 [0.         0.         0.14174569]]
prior_mean: [0. 0. 0.]

cov matrix det: 0.0017095304850218863


In [12]:
print_attributes(ds.means_vars[1])
print()
print("cov matrix det:" , np.linalg.det(ds.means_vars[0].cov))

cov: [[0.1632848  0.         0.        ]
 [0.         0.07386203 0.        ]
 [0.         0.         0.14174569]]
d: 3
k: 1
mean: [0.8844881  0.08486708 0.32211674]
prior_cov: [[0.1632848  0.         0.        ]
 [0.         0.07386203 0.        ]
 [0.         0.         0.14174569]]
prior_mean: [0. 0. 0.]

cov matrix det: 0.0017095304850218863


## γ variables

In [13]:
print_attributes(ds.gamma_vars[0])
print()
print("cov matrix det:" , np.linalg.det(ds.gamma_vars[0].cov))

cov: [[0.00438215 0.        ]
 [0.         0.01640487]]
d: 3
dim: 2
k: 0
mean: [ 0.03146241 -0.19976272]
nu: 0.02214891160027294
prior_cov: [[0.00438215 0.        ]
 [0.         0.01640487]]

cov matrix det: 7.188850842331236e-05


In [14]:
print_attributes(ds.gamma_vars[1])
print()
print("cov matrix det:" , np.linalg.det(ds.gamma_vars[1].cov))

cov: [[0.00438215 0.        ]
 [0.         0.01640487]]
d: 3
dim: 2
k: 1
mean: [-0.10038209  0.00028987]
nu: 0.05498130332112595
prior_cov: [[0.00438215 0.        ]
 [0.         0.01640487]]

cov matrix det: 7.188850842331236e-05


## sigma star variables

In [15]:
print_attributes(ds.sigma_star_vars[0])

print("sigma_star_scale_det:", np.linalg.det(ds.sigma_star_vars[0].scale))

d: 3
dim: 2
dof: 6
first_moment: None
k: 0
nu: 0.02214891160027294
prior_dof: 6
prior_scale: [[ 0.04483136 -0.03819025]
 [-0.03819025  0.08475071]]
scale: [[ 0.04940718 -0.01925336]
 [-0.01925336  0.03124995]]
sigma_star_scale_det: 0.0011732802877506877


In [16]:
print_attributes(ds.sigma_star_vars[1])

print("sigma_star_scale_det:", np.linalg.det(ds.sigma_star_vars[1].scale))


d: 3
dim: 2
dof: 6
first_moment: None
k: 1
nu: 0.05498130332112595
prior_dof: 6
prior_scale: [[ 0.04483136 -0.03819025]
 [-0.03819025  0.08475071]]
scale: [[ 0.00497235 -0.01124041]
 [-0.01124041  0.12002038]]
sigma_star_scale_det: 0.00047043630574278576


In [17]:
ds.gmm.print_labels()

Number of elements in each group:
Group 0: 326 elements
Group 1: 322 elements
Group 2: 352 elements


In [18]:
ds.dataset_vi(max_iter=10)

[[0.12389962 0.53353438 0.7802318 ]
 [0.74157919 0.57118574 0.03513042]
 [0.88411673 0.0813276  0.32531048]] cluster centres
self.gamma_estimates:  [[ 0.02937188 -0.1996718 ]
 [-0.04291738  0.03761332]
 [-0.10068865  0.00249248]]
mu_cov 0 det: 0.0017116786547988717
scale_mat 0 det: 0.002349325653139476
gamma_cov 0 det: 6.96483247587759e-05
gamma cov: [[0.0042465  0.        ]
 [0.         0.01640135]]
mu_cov 1 det: 0.0017116786547988717
scale_mat 1 det: 0.002349325653139476
gamma_cov 1 det: 6.96483247587759e-05
gamma cov: [[0.0042465  0.        ]
 [0.         0.01640135]]
mu_cov 2 det: 0.0017116786547988717
scale_mat 2 det: 0.002349325653139476
gamma_cov 2 det: 6.96483247587759e-05
gamma cov: [[0.0042465  0.        ]
 [0.         0.01640135]]


Performing VI:   0%|          | 0/10 [00:00<?, ?it/s]


TypeError: return arrays must be of ArrayType

In [23]:
ds.means_vars[1].mean

array([0.02443844, 0.01397415, 0.05581803])

In [22]:
ds.mean_vars[1].mean

AttributeError: 'Synthetic_data' object has no attribute 'mean_vars'